In [ ]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
def load_event(filepath, provider):
  if provider in ['bitfinex']:
    try:
      l = pd.read_csv(filepath, header=None)
    except:
      msg = f'File not found!'
      print (msg)
      return msg
    if provider == 'bitfinex':
      l[1] = l[1].str[1:]
      l[3] = l[3].str[:-1].astype(float)
      l[5] = l[5].str[:-1]
      l[5] = pd.to_datetime(l[5], unit='ms')
      l = l[[5,1,2,3]]
      l.columns = ['timestamp','orderId','price','volume']
    return l
  else:
    msg = f'Not support data provider "{provider}" format'
    print (msg)
    return msg

In [ ]:
data = load_event('/content/drive/MyDrive/WQU-Capstone-HFT/Dataset/Kaiko/L3_OrderBooks_Extract/bitfinex_tbtcusd_2020-06-15_event.csv','bitfinex')

In [ ]:
data[:]

,timestamp,orderId,price,volume
0,2020-06-15 00:00:00.022,46471382001,9340.7,0.030770
1,2020-06-15 00:00:00.022,46471382020,9340.7,0.023495
2,2020-06-15 00:00:00.022,46470902536,9347.9,-0.030770
3,2020-06-15 00:00:00.022,46470902572,9349.4,-0.017508
4,2020-06-15 00:00:01.801,46471381985,9341.6,0.017508
...,...,...,...,...
4404105,2020-06-15 23:59:59.890,46513904061,9440.3,-0.081036
4404106,2020-06-15 23:59:59.890,46514020886,9442.7,-0.018552
4404107,2020-06-15 23:59:59.961,46512616017,9434.9,0.004200
4404108,2020-06-15 23:59:59.961,46514020883,9434.9,0.018552


In [ ]:
# delete orderId which appear only one
data = data[data.orderId.isin(list(data.groupby('orderId').size()[data.groupby('orderId').size()>=2].index))]
data = data.reset_index(drop=True)

In [ ]:
# data.groupby('orderId')['price'].min()[data.groupby('orderId')['price'].min() > 0]

In [ ]:
# data[data.orderId == '46471382001']  #.iloc[687,:]#[data.orderId == '46471382021']

In [ ]:
l3book = pd.DataFrame(columns=['timestamp','bid_1_price','bid_1_vol','bid_2_price','bid_2_vol','bid_3_price','bid_3_vol','bid_4_price','bid_4_vol','bid_5_price','bid_5_vol', \
                               'ask_1_price','ask_1_vol','ask_2_price','ask_2_vol','ask_3_price','ask_3_vol','ask_4_price','ask_4_vol','ask_5_price','ask_5_vol'])

In [ ]:
l_timestamp = None
t_order = {}
t_book = {}

for index, r in data.iterrows():
  if t_order.get(r['orderId']):
    if r['price'] == 0:
      if t_book.get(t_order[r['orderId']]['price']):
        t_book[t_order[r['orderId']]['price']] -= t_order[r['orderId']]['volume']  
    else:
      if t_order[r['orderId']]['price'] != 0:
        t_book[t_order[r['orderId']]['price']] -= t_order[r['orderId']]['volume']

      if t_book.get(r['price']):
        t_book[r['price']] += r['volume']
      else:
        t_book[r['price']] = r['volume']
  else:
    if r['price'] != 0:
      if t_book.get(r['price']):
        t_book[r['price']] += r['volume']
      else:
        t_book[r['price']] = r['volume']

  t_order[r['orderId']]= {'price': r['price'], 'volume': r['volume']}


  if r['timestamp'] != data.iloc[index+1,:]['timestamp']:

    dl = []
    pl = []
    nl = []

    for k,v in t_book.items():
      v = np.round(v,12)
      if v == 0:
        dl.append(k)
      elif v > 0:
        pl.append(k)
      elif v<0:
        nl.append(k)
    if len(dl)>0:
      for e in dl:
        del t_book[e]
      del dl

    row = {'timestamp':0,'bid_1_price':0,'bid_1_vol':0,'bid_2_price':0,'bid_2_vol':0,'bid_3_price':0,'bid_3_vol':0,'bid_4_price':0,'bid_4_vol':0,'bid_5_price':0,'bid_5_vol':0,
          'ask_1_price':0,'ask_1_vol':0,'ask_2_price':0,'ask_2_vol':0,'ask_3_price':0,'ask_3_vol':0,'ask_4_price':0,'ask_4_vol':0,'ask_5_price':0,'ask_5_vol':0}
    row['timestamp'] = r['timestamp']

    try:
      t_bid = sorted(pl, reverse=True)[:5]
      row['bid_1_price'] = t_bid[0]
      row['bid_1_vol'] = t_book.get(t_bid[0])
      row['bid_2_price'] = t_bid[1]
      row['bid_2_vol'] = t_book.get(t_bid[1])
      row['bid_3_price'] = t_bid[2]
      row['bid_3_vol'] = t_book.get(t_bid[2])
      row['bid_4_price'] = t_bid[3]
      row['bid_4_vol'] = t_book.get(t_bid[3])
      row['bid_5_price'] = t_bid[4]
      row['bid_5_vol'] = t_book.get(t_bid[4])
      del t_bid
    except:
      pass
    try:
      t_ask = sorted(nl, reverse=False)[:5]
      row['ask_1_price'] = t_ask[0]
      row['ask_1_vol'] = t_book.get(t_ask[0])
      row['ask_2_price'] = t_ask[1]
      row['ask_2_vol'] = t_book.get(t_ask[1])
      row['ask_3_price'] = t_ask[2]
      row['ask_3_vol'] = t_book.get(t_ask[2])
      row['ask_4_price'] = t_ask[3]
      row['ask_4_vol'] = t_book.get(t_ask[3])
      row['ask_5_price'] = t_ask[4]
      row['ask_5_vol'] = t_book.get(t_ask[4])
      del t_ask
    except:
      pass

    if r['timestamp'] != l_timestamp:
      l3book = l3book.append(row, ignore_index=True)
    else:
      l3book.iloc[-1,:] = row

    l_timestamp = r['timestamp']

  if index % 2000 == 0:
    l3book.to_csv('/content/drive/MyDrive/WQU-Capstone-HFT/Dataset/Kaiko/L3_OrderBooks_Extract/l5book_2.csv')
  if index % 10000 == 0:
    print(f'{index} at {datetime.now()}')

0 at 2021-04-08 10:25:45.779394
10000 at 2021-04-08 10:26:05.236165
20000 at 2021-04-08 10:26:24.647349
30000 at 2021-04-08 10:26:54.181311
40000 at 2021-04-08 10:27:30.409350
50000 at 2021-04-08 10:28:17.292599
60000 at 2021-04-08 10:29:14.439696
70000 at 2021-04-08 10:30:22.165681
80000 at 2021-04-08 10:32:00.587110
90000 at 2021-04-08 10:34:25.853796
100000 at 2021-04-08 10:37:01.468744
110000 at 2021-04-08 10:38:47.955968
120000 at 2021-04-08 10:41:20.779422
130000 at 2021-04-08 10:44:49.226946
140000 at 2021-04-08 10:48:02.696236
150000 at 2021-04-08 10:52:15.594632
160000 at 2021-04-08 10:56:49.241622
170000 at 2021-04-08 10:59:55.635908
180000 at 2021-04-08 11:03:58.198491
190000 at 2021-04-08 11:07:58.810618
200000 at 2021-04-08 11:12:35.247460
210000 at 2021-04-08 11:15:45.389715
220000 at 2021-04-08 11:19:01.903244
230000 at 2021-04-08 11:23:27.920341
240000 at 2021-04-08 11:27:38.033641
250000 at 2021-04-08 11:31:39.374698
260000 at 2021-04-08 11:36:28.280830
270000 at 2021-

In [ ]:
l3book

In [ ]:
l3book.to_csv('l3book.csv')
l3book.to_csv('/content/drive/MyDrive/WQU-Capstone-HFT/Dataset/Kaiko/L3_OrderBooks_Extract/l3book.csv')

In [ ]:
l3book[(l3book.bid_1_price > l3book.ask_1_price)]

In [ ]:
data.groupby('orderId')['orderId'].size().sort_values()[35:50]
# data[data.orderId == '46502760279']

In [ ]:
import sys

print(sys.getsizeof(data))
print(sys.getsizeof(data.values.tolist()))

### temp

l_timestamp = None
t_order = {}
t_book = {'bid': {}, 'ask': {}}
for index, r in data.iterrows():
  if r['timestamp'] != l_timestamp:
    if len(l3book) > 0:
      t_row = l3book.iloc[-1,:].to_dict()
      bid_list = [t_row['bid_1_price'],t_row['bid_2_price'],t_row['bid_3_price']]
      ask_list = [t_row['ask_1_price'],t_row['ask_2_price'],t_row['ask_3_price']]
      t_row['timestamp'] = r['timestamp']
      if r['price'] == 0:
        if r['volume'] == 1:
          if r['price'] in bid_list:
            pass
        else:
          pass
        del t_order[r['orderId']]
      l3book = l3book.append(t_row, ignore_index=True)
    else:
      t_row = {'timestamp':r['timestamp']}
      if r['volume'] > 0:
        t_row['bid_1_price'] = r['price']
        t_row['bid_1_vol'] = r['volume']
        if t_book['bid'].get(r['price']):
          t_book['bid'][r['price']] += r['volume']
        else:
          t_book['bid'][r['price']] = r['volume']
      else:
        t_row['ask_1_price'] = r['price']
        t_row['ask_1_vol'] = r['volume']
        if t_book['ask'].get(r['price']):
          t_book['ask'][r['price']] += r['volume']
        else:
          t_book['ask'][r['price']] = r['volume']
      t_order[r['orderId']]= {'price': r['price'], 'volume': r['volume']}
      l3book = l3book.append(t_row, ignore_index=True)
    l_timestamp = r['timestamp']
  if index > 100:
    break

l_timestamp = None
t_order = {}
t_book = {'bid': {}, 'ask': {}}
for index, r in data.iterrows():
  if r['price'] == 0:
    if t_order.get(r['orderId']):
      if r['volume'] == 1:
        if t_book['bid'].get(t_order[r['orderId']]['price']):
          t_book['bid'][t_order[r['orderId']]['price']] -= r['volume']
        else:
          t_book['bid'][t_order[r['orderId']]['price']] = 0
      else:
        if t_book['ask'].get(t_order[r['orderId']]['price']):
          t_book['ask'][t_order[r['orderId']]['price']] -= r['volume']
        else:
          t_book['ask'][t_order[r['orderId']]['price']] = 0
      if t_order.get(r['orderId']):
        del t_order[r['orderId']]
      if t_book['bid'].get(0):
        del t_book['bid'][0]
      if t_book['ask'].get(0):
        del t_book['ask'][0]      
  else:
    if r['volume'] > 0:
      if t_book['bid'].get(r['price']):
        t_book['bid'][r['price']] += r['volume']
      else:
        t_book['bid'][r['price']] = r['volume']
    else:
      if t_book['ask'].get(r['price']):
        t_book['ask'][r['price']] += r['volume']
      else:
        t_book['ask'][r['price']] = r['volume']    
    t_order[r['orderId']]= {'price': r['price'], 'volume': r['volume']}

  if 0 in list(t_book['bid'].values()):
    dl = []
    for k,v in t_book['bid'].items():
      if v == 0:
        dl.append(k)
    if len(dl)>0:
      for e in dl:
        del t_book['bid'][e]
      del dl    

  if 0 in list(t_book['ask'].values()):
    dl = []
    for k,v in t_book['ask'].items():
      if v == 0:
        dl.append(k)
    if len(dl)>0:
      for e in dl:
        del t_book['ask'][e]
      del dl

  row = {'timestamp':0,'bid_1_price':0,'bid_1_vol':0,'bid_2_price':0,'bid_2_vol':0,'bid_3_price':0,'bid_3_vol':0,
         'ask_1_price':0,'ask_1_vol':0,'ask_2_price':0,'ask_2_vol':0,'ask_3_price':0,'ask_3_vol':0}
  row['timestamp'] = r['timestamp']
  try:
    t_bid = list(reversed(sorted(t_book['bid'].items())))[:3]
    row['bid_1_price'] = t_bid[0][0]
    row['bid_2_price'] = t_bid[1][0]
    row['bid_3_price'] = t_bid[2][0]
    row['bid_1_vol'] = t_bid[0][1]
    row['bid_2_vol'] = t_bid[1][1]
    row['bid_3_vol'] = t_bid[2][1]
    t_ask = sorted(t_book['ask'].items())[:3]
    row['ask_1_price'] = t_ask[0][0]
    row['ask_2_price'] = t_ask[1][0]
    row['ask_3_price'] = t_ask[2][0]
    row['ask_1_vol'] = t_ask[0][1]
    row['ask_2_vol'] = t_ask[1][1]
    row['ask_3_vol'] = t_ask[2][1]
    del t_bid, t_ask
  except:
    pass

  if r['timestamp'] != l_timestamp:
    l3book = l3book.append(row, ignore_index=True)
  else:
    l3book.iloc[-1,:] = row

  l_timestamp = r['timestamp']

  if index >= 2000:
    break

In [ ]:
d = {1:'a',2:'b',3:'c'}
list(d.values())

In [ ]:
d = [5,20,10,12]
sorted(d, reverse=True)

In [ ]:
a = 0.9876512361
np.round(a,2)